In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [78]:
towerseasons = pd.read_csv('tower_seasons_2019-11-21.csv')
towerweekly = pd.read_csv('tower_weekly_2019-11-22.csv')
towers = pd.read_csv('towers_2019-11-21.csv')

In [79]:
towerseasons_clean = towerseasons.dropna()
towerweekly_clean = towerweekly.dropna().copy()
towers_clean = towers.dropna()

In [41]:
towerseasons_clean.head(2)

,tower_id,year,actual_max_canopy,max_canopy_date
2,3512,2019,0.458826,2019-07-08
13,825,2019,0.659543,2019-07-01


In [99]:
towerweekly_clean['year'] = pd.DatetimeIndex(towerweekly_clean['date']).year
towerweekly_clean.head(2)

,tower_id,date,irrigation_mm,actual_precip_mm,pri,year
0,10,2019-01-07,0.0,14.204270,1.183220,2019
1,10,2019-01-14,0.0,85.758867,1.158736,2019


In [ ]:
## Identifying PRI and max canopy values that don't match and filtering them out. Major cleaning step

In [81]:
# Step 1 - Narrow down to columns of interest
towerweekly_group_pri_clean= towerweekly_clean[['year', 'tower_id', 'pri']]
# towerweekly_group_pri_clean.head()

,year,tower_id,pri
0,2019,10,1.183220
1,2019,10,1.158736
2,2019,10,1.063871
3,2019,10,1.088352
4,2019,10,1.166881


In [ ]:
# Step 2 - Merge previous dataframe with tower_seasons 
tower_pri_comb = towerseasons_clean.merge(towerweekly_group_pri_clean, on =['tower_id', 'year'])
# tower_pri_comb.head()

In [82]:
# Step 3 - Create a new column 'different' which calculates difference between max cnaopy and PRI
# Remember we want values that are NOT different aka the difference should equal 0 
tower_pri_comb['different'] = tower_pri_comb.actual_max_canopy - tower_pri_comb.pri
# tower_pri_comb.head()

,tower_id,year,actual_max_canopy,max_canopy_date,pri
0,3512,2019,0.458826,2019-07-08,0.969149
1,3512,2019,0.458826,2019-07-08,0.923842
2,3512,2019,0.458826,2019-07-08,0.745649
3,3512,2019,0.458826,2019-07-08,0.898334
4,3512,2019,0.458826,2019-07-08,0.988596


In [84]:
# Step 5 - How many cases did NOT equal 0?
len(tower_pri_comb[tower_pri_comb['different'] != 0 ])

31345

In [85]:
# Step 6 - More importantly, how many cases did equal 0? Almost 600, which is about what we expected.
len(tower_pri_comb[tower_pri_comb['different'] == 0 ])

594

In [87]:
# Step 7 - This is your new dataframe 
trimmed_tower_weekly = tower_pri_comb[tower_pri_comb['different'] == 0 ].copy()
trimmed_tower_weekly.head()

,tower_id,year,actual_max_canopy,max_canopy_date,pri,different
26,3512,2019,0.458826,2019-07-08,0.458826,0.0
89,2048,2019,0.660164,2019-06-17,0.660164,0.0
187,832,2019,0.515444,2019-07-08,0.515444,0.0
205,4230,2019,0.580303,2019-07-08,0.580303,0.0
319,2305,2019,0.701330,2019-06-10,0.701330,0.0


In [114]:
# Step 8 - Do the same exact thing but add the other variables back in.
# Not deleting previous one because of paranoia and fear of losing it.
towerweekly_group_pri_clean_complete = towerweekly_clean[['year', 'tower_id', 'pri', 'irrigation_mm', 'actual_precip_mm', 'date']]
tower_pri_comb_complete = towerseasons_clean.merge(towerweekly_group_pri_clean_complete, on =['tower_id', 'year'])
tower_pri_comb_complete['different'] = tower_pri_comb_complete.actual_max_canopy - tower_pri_comb_complete.pri
len(tower_pri_comb_complete[tower_pri_comb_complete['different'] != 0 ])
len(tower_pri_comb_complete[tower_pri_comb_complete['different'] == 0 ])
trimmed_tower_weekly_complete = tower_pri_comb_complete[tower_pri_comb_complete['different'] == 0 ].copy()
trimmed_tower_weekly.head()
len(trimmed_tower_weekly)

594

In [88]:
# Groupby statement for relevant stats
towerweekly_group = trimmed_tower_weekly.groupby(['year', 'tower_id']).mean()
towerweekly_group.head()

actual_max_canopy       pri  different
year tower_id                                        
2019 10                 0.500498  0.500498        0.0
     18                 0.768882  0.768882        0.0
     20                 0.813847  0.813847        0.0
     21                 0.559340  0.559340        0.0
     27                 0.693847  0.693847        0.0

In [ ]:
# Lets merge our data to create one dataframe

In [115]:
print("Towers dimensions: {}".format(trimmed_tower_weekly_complete.shape))
print("Towers Weekly: {}".format(towerseasons_clean.shape))
print("Towers Seasonal: {}".format(towers_clean.shape))

Towers dimensions: (594, 9)
Towers Weekly: (4361, 4)
Towers Seasonal: (1242, 11)


In [90]:
trimmed_tower_weekly_complete.head(2)

,tower_id,year,actual_max_canopy,max_canopy_date,pri,different
26,3512,2019,0.458826,2019-07-08,0.458826,0.0
89,2048,2019,0.660164,2019-06-17,0.660164,0.0


In [116]:
# Merge previous table with towers on tower_id
# Is a fairly small data frame. Something to consider. 
final_table = pd.merge(trimmed_tower_weekly_complete, towers_clean, on = 'tower_id')
print("Towers Seasonal: {}".format(final_table.shape))

Towers Seasonal: (377, 19)


In [117]:
final_table.head()

,tower_id,year,actual_max_canopy,max_canopy_date,pri,irrigation_mm,actual_precip_mm,date,different,crop,metacrop,betweenrowft,slope,aspect,region,subregion,installed_at,young_plant,trellis_type
0,3512,2019,0.458826,2019-07-08,0.458826,0.000000,0.0,2019-07-08,0.0,grape,vine,7.0,2,198,NCoast,carneros,2018-06-07 22:24:00,False,quad
1,2048,2019,0.660164,2019-06-17,0.660164,2.020265,0.0,2019-06-17,0.0,grape,vine,8.0,1,206,CCoast,pasorobles,2017-03-09 19:09:41,True,vsp
2,832,2019,0.515444,2019-07-08,0.515444,0.000000,0.0,2019-07-08,0.0,grape,vine,8.0,2,190,NCoast,carneros,2016-03-03 00:25:00,False,vsp
3,4230,2019,0.580303,2019-07-08,0.580303,6.047731,0.0,2019-07-08,0.0,grape,vine,8.0,10,335,NCoast,carneros,2019-04-19 18:47:10,False,vsp
4,2305,2019,0.701330,2019-06-10,0.701330,0.000000,0.0,2019-06-10,0.0,grape,vine,9.0,1,225,CCoast,pasorobles,2017-04-26 22:30:47,True,vsp
